In [1]:
import os
from dotenv import load_dotenv, find_dotenv

import tiktoken
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

load_dotenv(find_dotenv(), override=True)

True

In [ ]:
# from getpass import getpass

# os.environ['OPENAI_API_KEY'] = getpass('Lutfen OpenAi api key giriniz:')

In [2]:
def load_document(file):
    name, ext = os.path.splitext(file)

    if ext == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'{file} yukleniyor..')
        loader = PyPDFLoader(file)
        return loader.load()
    elif ext == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'{file} yukleniyor..')
        loader = Docx2txtLoader(file)
        return loader.load()
    elif ext == '.txt':
        from langchain.document_loaders import TextLoader
        print(f'{file} yukleniyor..')
        loader = TextLoader(file, encoding='utf-8')
        return loader.load()
    else:
        print('Yuklenen dokuman PDF, DOCX veya TXT uzantili olmalidir.')
        return None

In [3]:
def get_num_tokens(string: str, encoder: str ="gpt-3.5-turbo") -> int:
    encoding = tiktoken.encoding_for_model(encoder)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [4]:
def summarizer_router(num_tokens: int) -> str:
    '''
    Kullanılan modelin gpt-3.5-turbo olduğu varsayılacaktır(16k context length) buna göre 'map_reduce' veya 'basic' döndürecektir.
    Sayılar başka bir model kullanılıyorsa ayarlanabilir.
    '''
    if num_tokens > 10000:
        return 'map_reduce'
    else:
        return 'basic'

In [5]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.1)

In [6]:
template = """Aşağıdaki metni özetle. Metindeki ana noktaları özetinde söyle. Özeti olabildiğince kısa ve öz tut.

###
{text}
###
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template
)

In [7]:
def basic_summarizer(text: str):
    input_prompt = prompt.format(text=text)
    ans = llm.invoke(input_prompt)
    return ans.content

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=500)

map_prompt = """Aşağıdaki metni kısa ve öz bir şekilde özetle:

###
{text}
###

Kısa ve öz özet:
"""

combine_prompt = """
Hashtagler içindeki metni kısa ve öz bir şekilde özetle.
Metindeki ana noktaları bul ve özetle beraber söyle.

###
{text}
###

Ana noktaların özeti:
"""

map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

def map_reduce_summarizer(text: str):
    docs = text_splitter.create_documents([text])
    summary_chain = load_summarize_chain(llm=llm, 
                                         chain_type='map_reduce',                                     
                                         map_prompt=map_prompt_template,
                                         combine_prompt=combine_prompt_template)
    return summary_chain.run(docs)

In [9]:
def summarize(text: str) -> str:
    num_tokens = get_num_tokens(text)
    router_decide = summarizer_router(num_tokens)
    if router_decide == 'basic':
        return basic_summarizer(text)
    else:
        return map_reduce_summarizer(text)

In [10]:
tck = load_document('./tck_bolum1.txt')[0].page_content

./tck_bolum1.txt yukleniyor..


In [11]:
summary = summarize(tck)
print(summary)

C:\Users\baran\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


- Ceza Kanunu suç işlenmesini önlemeyi ve kişi haklarını korumayı amaçlar.
- Kanunun temel esasları suç ve ceza türlerini düzenler.
- Adalet ve eşitlik ilkesi kanunun bağlayıcılığını vurgular.
- Suçun oluşması kastın varlığına bağlıdır.
- Hapis cezasının ertelenmesi, adli para cezası, hakların yoksun bırakılması gibi güvenlik tedbirleri konuları ele alınır.
- Dava zamanaşımı süreleri belirli ceza türleri için geçerlidir.
- Ceza zamanaşımı ve hak yoksunlukları ceza zamanaşımı dolana kadar devam eder.
- Müsaderede zamanaşımı 20 yıl sonra infaz edilmez.
- Soruşturulması ve kovuşturulması şikâyete bağlı suçlarda şikâyet hakkı olan kişinin altı ay içinde şikâyette bulunmaması durumunda soruşturma ve kovuşturma yapılamaz.


In [12]:
capablanca = '''
José Raúl Capablanca y Graupera (19 Kasım 1888 - 8 Mart 1942) Kübalı dünya satranç şampiyonu. 
1921 ile 1927 yılları arasında dünya satranç şampiyonu unvanını taşımıştır. 
1921 yılında Emanuel Lasker'i Havana'da yenerek unvanı kazanmış, 
1927'de Alexander Alekhine'e yenilerek bu unvanı devretmiştir.

Satranca 4 yaşında başlamıştır. 12 yaşında Küba şampiyonu J.Corzo'yu yenerek Küba şampiyonluğunu kazanmıştır. 
21 yaşında Amerikan şampiyonu Marshall'ı yenmiştir. 1911 San Sebastian uluslararası satranç turnuvasını kazanmıştır. 
1914 yılında Petersburg uluslararası satranç turnuvasında Emanuel Lasker'in yarım puan arkasından ikinci olmuştur. 
I. Dünya Savaşı'nın başlaması ile beraber Emanuel Lasker ile şampiyonluk unvan maçı ertelenmiş ve ancak 1921 yılında oynanabilmiştir. 
4 galibiyet, 10 beraberlik sonrası ve maç kaybetmeksizin Lasker'den şampiyonluk unvanını almıştır. 
Londra 1922 turnuvasını ezici bir üstünlükle kazanmıştır. 
Ancak 1924 New York turnuvasında ikinci, 1925 Moskova turnuvasında üçüncü olur. 
Bu dönemde otörler Capablanca'nın zirveye ulaştıktan sonra artık ilerlemeye ve oyununu geliştirmeye yönelik itici gücün kaybolduğu konusunda görüş bildirirler.
Capablanca'nın zaten satrançla ilgili çok çalışmadığı ve doğal yeteneği ile bu başarıları kazandığı söylenir. 
Aldığı üçüncülükten sonra daha sıkı çalışmaya başlar ve 1927 New York turnuvasını bütün maçları kazanarak kazanır. 
İkinci olan Alexander Alekhine Capablanca ile şampiyonluk unvan maçı oynamaya hak kazanır.
'''

In [13]:
summary = summarize(capablanca)
print(summary)

José Raúl Capablanca, Kübalı dünya satranç şampiyonudur. 1921-1927 yılları arasında unvanı taşımıştır. Satrancı 4 yaşında öğrenmiş, genç yaşta birçok başarı elde etmiştir. Capablanca'nın doğal yeteneği ve az çalışmasıyla başarıları kazandığı söylenir. 1927'de Alexander Alekhine'e yenilerek unvanı kaybetmiştir.
